# Import Libraries

In [83]:
import os
import numpy as np
import cv2
import openpyxl
from pyzbar.pyzbar import decode
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Load the dataset

In [84]:
# Function to load and preprocess images
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.resize(img, (100, 100))  # Resize image to desired dimensions
        if img is not None:  # Check if the image is valid
            images.append(img)
            label = filename.split('_')[0]  # Extract label from filename
            labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess dataset
dataset_folder = 'qr_dataset'  # Path to your dataset folder
images, labels = load_images(dataset_folder)

if len(images) > 0:
    print("Dataset loaded successfully.")
    print("Number of images:", len(images))
else:
    print("Failed to load the dataset. Please check the dataset folder path.")

Dataset loaded successfully.
Number of images: 1429


In [85]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Dataset Splitting

In [86]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

In [87]:
# Convert labels to categorical
num_classes = len(np.unique(labels_encoded))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Build CNN model for feature extraction

In [88]:
# Build CNN model for feature extraction
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Normalize pixel values to be between 0 and 1

In [89]:
# Normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [90]:
# Compile the CNN model
cnn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model

In [91]:
# Train the CNN model
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
33/33 [==============================] - 9s 257ms/step - loss: 2.0464 - accuracy: 0.7315 - val_loss: 1.6686 - val_accuracy: 0.7304
Epoch 2/10
33/33 [==============================] - 8s 250ms/step - loss: 1.4505 - accuracy: 0.7539 - val_loss: 1.7561 - val_accuracy: 0.7304
Epoch 3/10
33/33 [==============================] - 8s 249ms/step - loss: 1.4093 - accuracy: 0.7539 - val_loss: 1.7945 - val_accuracy: 0.7304
Epoch 4/10
33/33 [==============================] - 8s 236ms/step - loss: 1.3753 - accuracy: 0.7539 - val_loss: 1.7542 - val_accuracy: 0.7304
Epoch 5/10
33/33 [==============================] - 8s 239ms/step - loss: 1.3368 - accuracy: 0.7549 - val_loss: 2.0476 - val_accuracy: 0.7304
Epoch 6/10
33/33 [==============================] - 8s 233ms/step - loss: 1.2663 - accuracy: 0.7549 - val_loss: 2.2337 - val_accuracy: 0.7304
Epoch 7/10
33/33 [==============================] - 8s 234ms/step - loss: 1.1853 - accuracy: 0.7539 - val_loss: 2.3582 - val_accuracy: 0.7304
Epoch 

# Save the CNN model

In [92]:
# Save the CNN model
cnn_model.save('cnn_model.h5')

C:\Users\anshu\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Use CNN for feature extraction

In [93]:
# Use CNN for feature extraction
cnn_features_train = cnn_model.predict(X_train)
cnn_features_test = cnn_model.predict(X_test)

9/9 [==============================] - 1s 58ms/step


# SVM model Train

In [94]:
# SVM model Train
svm_model = SVC(kernel='linear')
svm_model.fit(cnn_features_train, np.argmax(y_train, axis=1))

SVC(kernel='linear')

# Save the SVM model

In [95]:
# Save the SVM model
import joblib
joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

# SVM Model Test

In [96]:
# SVM Model Test
svm_pred = svm_model.predict(cnn_features_test)

# Evaluate the model

In [97]:
# Evaluate the model
accuracy = accuracy_score(np.argmax(y_test, axis=1), svm_pred)
print('Test accuracy:', accuracy)

Test accuracy: 0.7552447552447552


# Classification report

In [98]:
# Classification report
print(classification_report(np.argmax(y_test, axis=1), svm_pred))

              precision    recall  f1-score   support

          23       0.00      0.00      0.00         1
          29       0.00      0.00      0.00         1
          30       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         1
          43       0.00      0.00      0.00         1
          44       0.00      0.00      0.00         1
          49       0.00      0.00      0.00         1
          51       0.00      0.00      0.00         1
          56       0.00      0.00      0.00         1
          58       0.00      0.00      0.00         1
          63       0.00      0.00      0.00         1
          65       0.00      0.00      0.00         1
          67       0.00      0.00      0.00         1
          70       0.00      0.00      0.00         1
          76       0.00      0.00      0.00         1
          78       0.00      0.00      0.00         1
          81       0.00      0.00      0.00         1
          86       0.00    

C:\Users\anshu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anshu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anshu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Real-time QR Code Attendance Tracking System Using Python

In [82]:
# Function to update Excel file with QR code data
def update_excel(qr_data):
    try:
        workbook = openpyxl.load_workbook('qr_code_attendance.xlsx')
    except FileNotFoundError:
        workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append([qr_data])  # Append QR code data to the Excel sheet
    workbook.save('qr_code_attendance.xlsx')

# Initialize variables to store the last scanned QR code data
last_scanned_qr_code = None

# Initialize camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Decode QR code
    decoded_objects = decode(frame)
    
    for obj in decoded_objects:
        qr_data = obj.data.decode('utf-8')
        
        # Check if QR code is different from the last scanned QR code
        if qr_data != last_scanned_qr_code:
            update_excel(qr_data)
            print(f"QR Code Data: {qr_data}")
            last_scanned_qr_code = qr_data
    
    cv2.imshow('QR Code Scanner', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


QR Code Data: 1031
QR Code Data: 1026
QR Code Data: 1002
QR Code Data: 1034
QR Code Data: 1015
